# DCC GARCH
Dynamic conditional correlation (DCC) generalized autoregressive conditional heteroskedastic (GARCH) model  
This is required to estimate the *marginal expected shortfall* (MES) and from that SRISK.
### `ccgarch`
https://cran.r-project.org/web/packages/ccgarch/  
https://github.com/cran/ccgarch    
https://www.r-project.org/conferences/useR-2008/slides/Nakatani.pdf  
### `rmgarch`
https://cran.r-project.org/web/packages/rmgarch/     
http://www.unstarched.net/2013/01/03/the-garch-dcc-model-and-2-stage-dccmvt-estimation/    
(Unable to install `rmgarch` on Azure.)

In [ ]:
# install.packages("rugarch")

In [ ]:
if(!require(ccgarch)){
    install.packages("ccgarch")
    library(ccgarch)
}

# Package `ccgarch` sample code

## Simulation - `dcc.sim`
Simulating data from the original DCC-GARCH(1,1) process  
See **P11** of https://cran.r-project.org/web/packages/ccgarch/ccgarch.pdf 

In [17]:
  nobs <- 50
  a <- c(0.003, 0.005, 0.001)
  A <- diag(c(0.2,0.3,0.15))
  B <- diag(c(0.75, 0.6, 0.8))
  uncR <- matrix(c(1.0, 0.4, 0.3, 0.4, 1.0, 0.12, 0.3, 0.12, 1.0),3,3)
  dcc.para <- c(0.01,0.98)
  dcc.data <- dcc.sim(nobs, a, A, B, uncR, dcc.para, model="diagonal")

In [ ]:
dcc.data

## Estimation - `dcc.estimation`
Estimating a DCC-GARCH(1,1) model  
See **P6** of https://cran.r-project.org/web/packages/ccgarch/ccgarch.pdf  

In [19]:
dcc.results <- dcc.estimation(inia=a, iniA=A, iniB=B, ini.dcc=dcc.para, 
        dvar=dcc.data$eps, model="extended")  # or model="diagonal"

****************************************************************
*  Estimation has been completed.                              *
*  The outputs are saved in a list with components:            *
*    out    : the estimates and their standard errors          *
*    loglik : the value of the log-likelihood at the estimates *
*    h      : a matrix of estimated conditional variances      *
*    DCC    : a matrix of DCC estimates                        *
*    std.resid : a matrix of the standardised residuals        *
*    first  : the results of the first stage estimation        *
*    second : the results of the second stage estimation       *
****************************************************************


In [24]:
head(dcc.results$h)

0.03858887,0.03777195,0.01315452
0.02407145,0.03777029,0.01247283
0.02262672,0.03776891,0.01355537
0.02794056,0.03776846,0.01420051
0.02821586,0.03776830,0.01641429
0.02550847,0.03776820,0.01500712


#### DCC estimates - `dcc.results$DCC`

In [25]:
head(dcc.results$DCC)

1,0.4832593,0.3501419,0.4832593,1,0.4116337,0.3501419,0.4116337,1
1,0.4832593,0.3501419,0.4832593,1,0.4116337,0.3501419,0.4116337,1
1,0.4832593,0.3501419,0.4832593,1,0.4116337,0.3501419,0.4116337,1
1,0.4832593,0.3501419,0.4832593,1,0.4116337,0.3501419,0.4116337,1
1,0.4832593,0.3501419,0.4832593,1,0.4116337,0.3501419,0.4116337,1
1,0.4832593,0.3501419,0.4832593,1,0.4116337,0.3501419,0.4116337,1


In [44]:
dim(dcc.results$DCC)

[1] 50  9

In [26]:
dcc.results$DCC[1,]

[1] 1.0000000 0.4832593 0.3501419 0.4832593 1.0000000 0.4116337 0.3501419
[8] 0.4116337 1.0000000

In [27]:
matrix(dcc.results$DCC[1,],nrow = 3,ncol = 3)

1.0000000,0.4832593,0.3501419
0.4832593,1.0000000,0.4116337
0.3501419,0.4116337,1.0000000


In [32]:
matrix(dcc.results$DCC[1,],nrow = 3,ncol = 3)

1.0000000,0.4832593,0.3501419
0.4832593,1.0000000,0.4116337
0.3501419,0.4116337,1.0000000


In [46]:
array(dcc.results$DCC, dim = c(50,3,3))[1,,]

1.0000000,0.4832593,0.3501419
0.4832593,1.0000000,0.4116337
0.3501419,0.4116337,1.0000000


# MES & SRISK for systemic risk

https://vlab.stern.nyu.edu/analysis/RISK.USFIN-MR.MES  

## Benoit Sylvain - Comparison of measures  - Matlab / Octave 
http://www.runmycode.org/companion/view/175  
https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1973950  

### `call_fct`

In [ ]:
# % Returns for the market index & the given asset
data = [index asset];
data = data(~isnan(data(:,2).*data(:,1)),:); #% select only elements which are not NaN
data_center = data - ones(size(data,1),1)*mean(data); #%demeaned returns

# %% GJR-GARCH and DCC
[parameters, loglikelihood, Ht, Rt, Qt, stdresid, likelihoods, stderrors, A,B, jointscores, H]
    =dcc_mvgarch(data_center,1,1,1,1);
ht_m=sqrt(H(:,1)); #%market conditional volatility
ht_i=sqrt(H(:,2)); #%asset conditional volatility
rho=squeeze(Rt(1,2,:)); #%conditional correlation
c = quantile(data_center(:,1),alpha); # % HS VaR (nonparametric), it's our systemic event, it's a scalar here
MES = - fct_MES(data_center,c,ht_m,ht_i,rho);
LRMES = (1-exp(-18.*MES)); # %without simulation
SRISK = k.*LTQ - (1-k).*(1-LRMES).*MV;

### `dcc_mvgarch`

#### Purpose:

Estimates a multivariate GARCH model using the DCC estimator of Engle and Sheppard

#### Usage:

In [ ]:
[parameters, loglikelihood, Ht, Rt, Qt, likelihoods, stdresid, stderrors, A,B, jointscores] 
    = dcc_mvgarch(data,dccQ,dccP,archQ,garchP)

#### Inputs:

data          = A zero mean t by k (asset number) vector of residuals from some filtration [t by k]
dccQ          = The lag length of the innovation term in the DCC estimator (a scalar)
dccP          = The lag length of the lagged correlation matrices in the DCC estimator (a scalar)
archQ         = One of two things:     A scalar, q     in which case a p innovation model is estimated for each series
                                       A k by 1 vector in which case the ith series has innovation terms p=archP(i)
garchP        = One of two things:     A scalar, p     in which case a q GARCH lags is used in estimation for each series
                                       A k by 1 vector in which case the ith series has lagged variance terms q=archQ(i)

#### Outputs:

In [ ]:
parameters    = A vector of parameters estimated from the model of the form    
                    [GarchParams(1) GarchParams(2) ... GarchParams(k) DCCParams]    
                    where the Garch Parameters from each estimation are of the form    
                    [omega(i) alpha(i1) alpha(i2) ... alpha(ip(i)) beta(i1) beta(i2) ... beta(iq(i))]    
loglikelihood = The log likelihood evaluated at the optimum    
Ht            = A k by k by t array of conditional variances    
Rt            = A k by k by t array of Rt elements    
Qt            = A k by k by t array of Qt elements   
stdresid      = A [t by k] matrix of standardized residuals   
likelihoods   = the estimated likelihoods t by 1   
stderrors     = A length(parameters)^2 matrix of estimated correct standard errors   
A             = The estimated A from the robust standard errors    
B             = The estimated B from the standard errors   
jointscores   = The estimated scores of the likelihood t by length(parameters)   
H             = Conditional Volatility univariate   

### `fct_MES`

In [ ]:
function [MES] = fct_MES(data,c,ht_m,ht_i,rho)
    em=(data(:,1))./ht_m; %market first column
    xi=((data(:,2)./ht_i)-rho.*em)./ sqrt(1-rho.^2); %asset second column
    bwd=1*(size(data,1)^(-0.2)); # Scaillet's bwd p21, I put 1 instead of the standard deviation because our shocks are iid with unit variance
    K1=sum(em.*(normcdf(((c./ht_m)-em)./bwd)))./(sum(normcdf(((c./ht_m)-em)./bwd)));
    K2=sum(xi.*(normcdf(((c./ht_m)-em)./bwd)))./(sum(normcdf(((c./ht_m)-em)./bwd)));
    MES = (ht_i.*rho.*K1) + (ht_i.*sqrt(1-rho.^2).*K2);

## `TommasoBelluzzo/SystemicRisk` - Matlab / Octave

### `main_pro.m`

https://github.com/TommasoBelluzzo/SystemicRisk/blob/master/ScriptsProbabilistic/main_pro.m    

In [ ]:
#% Supply 2 series of returns (with means subtracted)
[p,s] = dcc_gjrgarch([ret0_m ret0_x]);
#% p       = An n-by-n-by-t matrix of floats containing the DCC coefficients.
#% s       = A t-by-n matrix of floats containing the conditional variances.
s_m = sqrt(s(:,1));
s_x = sqrt(s(:,2));
p_mx = squeeze(p(1,2,:)); #% Pull out the off-diagonal correlation (between market & firm)

beta_x = p_mx .* (s_x ./ s_m);
var_x = s_x * quantile((ret0_x ./ s_x),data.A); % Find the variance of the firm?

[mes,lrmes] = calculate_mes(ret0_m,s_m,ret0_x,s_x,beta_x,p_mx,data.A,data.D); #% Hopefully R package ccgarch can do this step?
srisk = calculate_srisk(lrmes,data.FrmsLia(:,i),data.FrmsCap(:,i),data.L); #% SRISK needs the (LR)MES + balance sheet data + crash level (e.g. 40%)

### `calculate_mes_internal`

https://github.com/TommasoBelluzzo/SystemicRisk/blob/master/ScriptsProbabilistic/calculate_mes.m

In [ ]:
function [mes,lrmes] = calculate_mes_internal(ret0_m,s_m,ret0_x,s_x,beta_x,p_mx,a,d)
    c = quantile(ret0_m,a);
    h = 1 * (length(ret0_m) ^ (-0.2));
    u = ret0_m ./ s_m;
    x_den = sqrt(1 - (p_mx .^ 2));
    x_num = (ret0_x ./ s_x) - (p_mx .* u);
    x = x_num ./ x_den;
    f = normcdf(((c ./ s_m) - u) ./ h);
    k1 = sum(u .* f) ./ sum(f);
    k2 = sum(x .* f) ./ sum(f);
    mes = (s_x .* p_mx .* k1) + (s_x .* x_den .* k2);
    lrmes = 1 - exp(log(1 - d) .* beta_x);

### `calculate_srisk_internal` 
https://github.com/TommasoBelluzzo/SystemicRisk/blob/master/ScriptsProbabilistic/calculate_srisk.m  

In [ ]:
function srisk = calculate_srisk_internal(lrmes,tl_x,mc_x,l)
    srisk = (l .* tl_x) - ((1 - l) .* (1 - lrmes) .* mc_x);
    srisk(srisk < 0) = 0;